# Compare model output with ASTE and observations

Load selected mooring time series (see choose_ASTE_profiles), and subsample CANARI LE accordingly

In [1]:
import xarray as xr
import numpy as np

Open all mooring time series and get grid points for subsetting CANARI-LE

In [2]:
import glob
mooring_files=glob.glob('../data/*tseries.nc')
mooring_vars={}
for file in mooring_files:
    mooring=xr.open_dataset(file)
    if 'iPROF' in mooring.dims:
        mooring=mooring.swap_dims({'iPROF':'time'})
    mooring=mooring.sortby('time').squeeze()
    if 'point' in mooring.dims:
        for ip,point in enumerate(mooring.point):
            mooring_vars[f'{mooring["loc"].values}_{ip}']=mooring[['prof_T','prof_Testim','prof_S','prof_Sestim']].sel(point=point)
    else:
        mooring_vars[str(mooring['loc'].values)]=mooring[['prof_T','prof_Testim','prof_S','prof_Sestim']]

Load NEMO T grid

In [3]:
import coast
nemo_dom = "/gws/nopw/j04/canari/users/dlrhodso/mesh_mask.nc"

config_grid={}
config_dir="../../tutorials/config"
for grid in ['t','f','u','v']:
    config_grid[grid]=f'{config_dir}/example_nemo_grid_{grid}.json'

nemo_t = coast.Gridded( fn_domain=nemo_dom, config=config_grid['t'])

/home/users/eboland/.conda/envs/canari-dask-mine/lib/python3.10/site-packages/pydap/lib.py:5: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
/home/users/eboland/.conda/envs/canari-dask-mine/lib/python3.10/site-packages/pkg_resources/__init__.py:2868: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pydap')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
/home/users/eboland/.conda/envs/canari-dask-mine/lib/python3.10/site-packages/pkg_resources/__init__.py:2868: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pydap.responses')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.h

Find NEMO grid points for all moorings

In [4]:
for mg_co in mooring_vars:
    mg_co_ds=mooring_vars[mg_co].squeeze()
    mean_lat=mg_co_ds.prof_lat.mean('time')
    mean_lon=mg_co_ds.prof_lon.mean('time')
    [mg_co_ds['yy'],mg_co_ds['xx']]=nemo_t.find_j_i(lat=mean_lat,lon=mean_lon)
    mooring_vars[mg_co]=mg_co_ds
    print(mg_co_ds.time.min('time').data,mg_co_ds.time.max('time').data)
mooring_vars

2005-08-30T00:00:00.000000000 2016-10-11T00:00:00.000000000
2007-09-02T00:00:00.000000000 2011-07-14T00:00:00.000000000
2004-09-27T00:00:00.000000000 2013-09-17T00:00:00.000000000
2002-08-13T00:00:00.000000000 2012-06-27T00:00:00.000000000
2014-08-12T00:00:00.000000000 2016-08-08T00:00:00.000000000
2014-08-12T00:00:00.000000000 2016-08-08T00:00:00.000000000


{'Beaufort Mooring': <xarray.Dataset>
 Dimensions:              (time: 3007)
 Coordinates: (12/21)
     prof_descr           (time) |S30 ...
     prof_date            (time) float64 ...
     prof_YYYYMMDD        (time) float64 ...
     prof_HHMMSS          (time) float64 ...
     prof_lon             (time) float64 -140.0 -140.0 -140.0 ... -140.1 -140.1
     prof_lat             (time) float64 74.0 74.0 74.0 74.0 ... 74.0 74.0 74.0
     ...                   ...
     prof_interp_lat      (time) float64 ...
     prof_interp_weights  (time) float64 ...
   * time                 (time) datetime64[ns] 2005-08-30 ... 2016-10-11
     depth                float64 ...
     loc                  <U16 'Beaufort Mooring'
     point                float64 1.134e+06
 Data variables:
     prof_T               (time) float64 ...
     prof_Testim          (time) float64 ...
     prof_S               (time) float64 ...
     prof_Sestim          (time) float64 ...
     yy                   int64 1114
   

In [5]:
%%time
import dask_gateway
import dask
# Create a connection to dask-gateway.
gw = dask_gateway.Gateway("https://dask-gateway.jasmin.ac.uk", auth="jupyterhub")

# Inspect and change the options if required before creating your cluster.
options = gw.cluster_options()
options.worker_cores = 1 #keeping this at 1 and allowing 15 worker processes seems to run faster than the other way around
options.scheduler_cores = 1 #we need at least one core for the scheduler
#specify which conda env to use, this must match the versions of python and dask (and a few other libraries) used on the notebook service
options.worker_setup='source /apps/jasmin/jaspy/mambaforge_envs/jaspy3.10/mf-22.11.1-4/bin/activate /gws/smf/j04/canari/dask-env'

# Create a dask cluster, or, if one already exists, connect to it.
# This stage creates the scheduler job in SLURM, so may take some time.
# While your job queues.
clusters = gw.list_clusters()
if not clusters:
    cluster = gw.new_cluster(options, shutdown_on_close=False)
else:
    cluster = gw.connect(clusters[0].name)

# Create at least one worker, and allow your cluster to scale to 15.
# The max JASMIN allows is 16, but one of these is used as the scheduler.
cluster.adapt(minimum=1, maximum=15)

# Get a dask client.
client = cluster.get_client()

CPU times: user 107 ms, sys: 29.4 ms, total: 136 ms
Wall time: 300 ms


Load all LE members

In [ ]:
data_path = "/gws/nopw/j04/canari/shared/large-ensemble/priority/HIST2"
ensembles=glob.glob(f'{data_path}/*')
def pre_proc(ds):
    return ds.drop_vars(['bounds_nav_lat','bounds_nav_lon','deptht_bounds','time_counter_bounds'])
    
for ens_file in ensembles:
    ens=int(ens_file.split('/')[-1])
    if glob.glob(f'../data/mooring_tseries_ens{ens}.nc'):
        print(f'File for {ens} exists, skipping')
        continue
    else:
        print(f'Doing ensemble member {ens}')
    ens_data_path=f'{data_path}/{ens}/OCN/yearly'
    Tdata=[]
    for year in range(2000,2015):
        temp_Tgrid = glob.glob(f"{ens_data_path}/{year}/*_mon__grid_T_votemper.nc")
        sal_Tgrid = glob.glob(f"{ens_data_path}/{year}/*_mon__grid_T_vosaline.nc")
        try:
            Tdata.append(xr.open_mfdataset([temp_Tgrid[0],sal_Tgrid[0]],preprocess=pre_proc))
        except:
            print(f'No data for member {ens}, year {year}')
            continue
    if Tdata:
        Tdata=xr.concat(Tdata,'time_counter')
    else:
        print(f'Tdata empty, skipping {ens} entirely')
        continue
    datetimeindex = Tdata.indexes['time_counter'].to_datetimeindex()
    Tdata['time_counter']=datetimeindex
    Tdata_allm=[]
    for loc in mooring_vars:
        mdata=mooring_vars[loc]
        Tdata_mooring=Tdata.sel(x=mdata.xx,y=mdata.yy)
        Tdata_mooring=Tdata_mooring.interp(deptht=mdata.depth)
        Tdata_allm.append(Tdata_mooring)
    Tdata_allm=xr.concat(Tdata_allm,'loc')
    print('Writing mooring t series to file')
    Tdata_allm.to_netcdf(f'../data/mooring_tseries_ens{ens}.nc')

File for 1 exists, skipping
File for 10 exists, skipping
File for 11 exists, skipping
File for 12 exists, skipping
File for 13 exists, skipping
File for 14 exists, skipping
Doing ensemble member 15


/tmp/ipykernel_1169/3974658622.py:28: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.


Writing mooring t series to file
Doing ensemble member 16


/tmp/ipykernel_1169/3974658622.py:28: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.


Writing mooring t series to file
Doing ensemble member 17


...

Writing mooring t series to file
Doing ensemble member 27


/tmp/ipykernel_1169/3974658622.py:28: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.


Writing mooring t series to file
File for 28 exists, skipping
Doing ensemble member 29


/tmp/ipykernel_1169/3974658622.py:28: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.


Writing mooring t series to file


In [36]:
data_path = "/gws/nopw/j04/canari/shared/large-ensemble/priority/SSP370"
ensembles=glob.glob(f'{data_path}/*')
def pre_proc(ds):
    return ds.drop_vars(['bounds_nav_lat','bounds_nav_lon','deptht_bounds','time_counter_bounds'])
    
for ens_file in ensembles:
    ens=int(ens_file.split('/')[-1])
    fout=f'../data/mooring_tseries_ens{ens}_ssp370.nc'
    if glob.glob(fout):
        print(f'File for {ens} exists, skipping')
        continue
    else:
        print(f'Doing ensemble member {ens}')
    ens_data_path=f'{data_path}/{ens}/OCN/yearly'
    Tdata=[]
    for year in range(2015,2025):
        temp_Tgrid = glob.glob(f"{ens_data_path}/{year}/*_mon__grid_T_votemper.nc")
        sal_Tgrid = glob.glob(f"{ens_data_path}/{year}/*_mon__grid_T_vosaline.nc")
        try:
            Tdata.append(xr.open_mfdataset([temp_Tgrid[0],sal_Tgrid[0]],preprocess=preproc))
        except:
            print(f'No data for member {ens}, year {year}')
            continue
    if Tdata:
        Tdata=xr.concat(Tdata,'time_counter')
    else:
        print(f'Tdata empty, skipping {ens} entirely')
        continue
    datetimeindex = Tdata.indexes['time_counter'].to_datetimeindex()
    Tdata['time_counter']=datetimeindex
    Tdata_allm=[]
    for loc in mooring_vars:
        mdata=mooring_vars[loc]
        Tdata_mooring=Tdata.sel(x=mdata.xx,y=mdata.yy)
        Tdata_mooring=Tdata_mooring.interp(deptht=mdata.depth)
        Tdata_allm.append(Tdata_mooring)
    Tdata_allm=xr.concat(Tdata_allm,'loc')
    print('Writing mooring t series to file')
    Tdata_allm.to_netcdf(fout)

File for 1 exists, skipping
File for 10 exists, skipping
File for 11 exists, skipping
File for 12 exists, skipping
File for 13 exists, skipping
File for 14 exists, skipping
File for 15 exists, skipping
File for 16 exists, skipping
File for 17 exists, skipping
File for 18 exists, skipping
File for 19 exists, skipping
File for 2 exists, skipping
File for 20 exists, skipping
File for 21 exists, skipping
File for 22 exists, skipping
File for 23 exists, skipping
File for 24 exists, skipping
File for 25 exists, skipping
File for 26 exists, skipping
File for 27 exists, skipping
File for 28 exists, skipping
File for 29 exists, skipping
File for 3 exists, skipping
File for 30 exists, skipping
File for 31 exists, skipping
File for 32 exists, skipping
File for 33 exists, skipping
File for 34 exists, skipping
File for 35 exists, skipping
File for 36 exists, skipping
File for 37 exists, skipping
File for 38 exists, skipping
File for 39 exists, skipping
File for 4 exists, skipping
File for 40 exists

Do the same for CMIP6 models from ISMIP, using HadGEM3-GC31-MM instead of UKESM

In [4]:
import sys
sys.path.insert(0,'/home/users/eboland/python')
import baspy as bp
import utils as ut
import pandas as pd
import xmip.preprocessing as pp #this makes all coordinates etc consistent across cmip6 models

In [5]:
def combined_preprocessing(ds):
    # fix naming
    ds = ut.rename_cmip6(ds)
    # promote empty dims to actual coordinates
    ds = pp.promote_empty_dims(ds)
    # demote coordinates from data_variables
    ds = pp.correct_coordinates(ds)
    # broadcast lon/lat
    ds = pp.broadcast_lonlat(ds)
    # shift all lons to consistent 0-360
    ds = pp.correct_lon(ds)
    # fix the units
    ds = pp.correct_units(ds)
    # rename the `bounds` according to their style (bound or vertex)
  # ds = parse_lon_lat_bounds(ds)
    # sort verticies in a consistent manner
   # ds = sort_vertex_order(ds)
    # convert vertex into bounds and vice versa, so both are available
   # ds = maybe_convert_bounds_to_vertex(ds)
   # ds = maybe_convert_vertex_to_bounds(ds)
   # ds = fix_metadata(ds)
   # ds = ds.drop_vars(_drop_coords, errors="ignore")
    return ds

In [6]:
models=['HadGEM3-GC31-MM','GFDL-ESM4','IPSL-CM6A-LR','MPI-ESM1-2-HR', 'MRI-ESM2-0']
#models=['MRI-ESM2-0',]
for model in models:
    print(model)
    for var in ['thetao','so']:
        df = bp.catalogue(dataset='cmip6',Var=[var,],Experiment=['historical',],CMOR='Omon',Model=model)
        for ir,row in df.iterrows():
        #    row=df.iloc[0]
            run=row.RunID
            if glob.glob(f'../data/mooring_tseries_{model}_{run}_{var}.nc'):
                print(f'File for {run} {var} exists, skipping')
                continue
            print(f'Doing {run} {var}')     
            files = bp.get_files(row)
            data=xr.open_mfdataset(files,preprocess=combined_preprocessing).sel(time=slice('2000-01-01',None))
            if not isinstance(data.indexes['time'], pd.DatetimeIndex):
                datetimeindex = data.indexes['time'].to_datetimeindex()
                data['time']=datetimeindex
            data_allm=[]
            for loc in mooring_vars:
                mdata=mooring_vars[loc]
                mlon=mdata.prof_lon.mean()
                mlat=mdata.prof_lat.mean()
                if mlon<0:
                    mlon=360+mlon
                [yy,xx]=ut.find_j_i(data,lon=mlon,lat=mlat)
                if type(xx)==np.int64:
                    data_mooring=data.isel(x=xx,y=yy)
                else:
                    data_mooring=data.isel(x=xx.compute(),y=yy.compute())
                data_mooring=data_mooring.interp(lev=mdata.depth)
                data_allm.append(data_mooring)
            data_allm=xr.concat(data_allm,'loc')
            data_allm=data_allm.assign_coords({'run':run,'model':model})
            print('Writing mooring t series to file')
            data_allm.to_netcdf(f'../data/mooring_tseries_{model}_{run}_{var}.nc')

HadGEM3-GC31-MM
Updating cached catalogue...
catalogue memory usage (MB): 26.848599
File for r1i1p1f3 thetao exists, skipping
File for r2i1p1f3 thetao exists, skipping
File for r3i1p1f3 thetao exists, skipping
File for r4i1p1f3 thetao exists, skipping
File for r1i1p1f3 so exists, skipping
File for r2i1p1f3 so exists, skipping
File for r3i1p1f3 so exists, skipping
File for r4i1p1f3 so exists, skipping
GFDL-ESM4
File for r1i1p1f1 thetao exists, skipping
File for r1i1p1f1 thetao exists, skipping
File for r1i1p1f1 so exists, skipping
File for r1i1p1f1 so exists, skipping
IPSL-CM6A-LR
File for r1i1p1f1 thetao exists, skipping
File for r2i1p1f1 thetao exists, skipping
File for r3i1p1f1 thetao exists, skipping
File for r4i1p1f1 thetao exists, skipping
File for r5i1p1f1 thetao exists, skipping
File for r6i1p1f1 thetao exists, skipping
File for r7i1p1f1 thetao exists, skipping
File for r8i1p1f1 thetao exists, skipping
File for r9i1p1f1 thetao exists, skipping
File for r1i1p1f1 so exists, skippi

/home/users/eboland/python/baspy/_catalogue.py:593: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  if len(pd.unique(list_file_extensions)) > 1:
/gws/nopw/j04/canari/users/emmomp/proj-Arctic-Ocean/notebooks/utils.py:65: UserWarning: rename 'j' to 'y' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  da = da.rename({di: target}).set_xindex(target)
/gws/nopw/j04/canari/users/emmomp/proj-Arctic-Ocean/notebooks/utils.py:65: UserWarning: rename 'i' to 'x' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  da = da.rename({di: target}).set_xindex(target)
/gws/nopw/j04/canari/users/emmomp/proj-Arctic-Ocean/notebooks/utils.py:65: UserWarning: rename 'j' to 'y' does not create an index anymore. Try using swap_dims instead or use set_index after

Writing mooring t series to file


/home/users/eboland/.conda/envs/canari-dask-mine/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 26.82 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


Doing r1i1p1f1 so


...

MRI-ESM2-0
Doing r1i1p1f1 thetao


/home/users/eboland/python/baspy/_catalogue.py:593: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  if len(pd.unique(list_file_extensions)) > 1:
/gws/nopw/j04/canari/users/emmomp/proj-Arctic-Ocean/notebooks/utils.py:83: UserWarning: CMIP.MRI.MRI-ESM2-0.historical.r1i1p1f1.Omon.gn.none.thetao:While renaming to target `lon_bounds`, more than one candidate was found ['x_bnds', 'vertices_longitude']. Renaming x_bnds to lon_bounds. Please double check results.
  warnings.warn(
/gws/nopw/j04/canari/users/emmomp/proj-Arctic-Ocean/notebooks/utils.py:83: UserWarning: CMIP.MRI.MRI-ESM2-0.historical.r1i1p1f1.Omon.gn.none.thetao:While renaming to target `lat_bounds`, more than one candidate was found ['y_bnds', 'vertices_latitude']. Renaming y_bnds to lat_bounds. Please double check results.
  warnings.warn(
/gws/nopw/j04/canari/users/emmomp/proj-Arctic-Ocean/notebooks/utils.py:83: UserWarning: C

Writing mooring t series to file
File for r1i1p1f1 thetao exists, skipping
Doing r1i1p1f1 so


/home/users/eboland/python/baspy/_catalogue.py:593: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  if len(pd.unique(list_file_extensions)) > 1:
/gws/nopw/j04/canari/users/emmomp/proj-Arctic-Ocean/notebooks/utils.py:83: UserWarning: CMIP.MRI.MRI-ESM2-0.historical.r1i1p1f1.Omon.gn.none.so:While renaming to target `lon_bounds`, more than one candidate was found ['x_bnds', 'vertices_longitude']. Renaming x_bnds to lon_bounds. Please double check results.
  warnings.warn(
/gws/nopw/j04/canari/users/emmomp/proj-Arctic-Ocean/notebooks/utils.py:83: UserWarning: CMIP.MRI.MRI-ESM2-0.historical.r1i1p1f1.Omon.gn.none.so:While renaming to target `lat_bounds`, more than one candidate was found ['y_bnds', 'vertices_latitude']. Renaming y_bnds to lat_bounds. Please double check results.
  warnings.warn(
/gws/nopw/j04/canari/users/emmomp/proj-Arctic-Ocean/notebooks/utils.py:83: UserWarning: CMIP.MRI.

Writing mooring t series to file
File for r1i1p1f1 so exists, skipping


Find EN4 observational data

In [3]:
en4_ds=xr.open_mfdataset('/gws/nopw/j04/nemo_vol1/jmecking/en4/v4.2.2/EN.4.2.2.f.analysis.g10.20*nc')
en4_ds

<xarray.Dataset>
Dimensions:                          (depth: 42, lat: 173, lon: 360, time: 288,
                                      bnds: 2)
Coordinates:
  * depth                            (depth) float32 5.022 15.08 ... 5.35e+03
  * lat                              (lat) float32 -83.0 -82.0 ... 88.0 89.0
  * lon                              (lon) float32 1.0 2.0 3.0 ... 359.0 360.0
  * time                             (time) datetime64[ns] 2000-01-16T12:00:0...
Dimensions without coordinates: bnds
Data variables:
    temperature                      (time, depth, lat, lon) float32 dask.array<chunksize=(1, 21, 87, 180), meta=np.ndarray>
    salinity                         (time, depth, lat, lon) float32 dask.array<chunksize=(1, 21, 87, 180), meta=np.ndarray>
    temperature_uncertainty          (time, depth, lat, lon) float32 dask.array<chunksize=(1, 21, 87, 180), meta=np.ndarray>
    salinity_uncertainty             (time, depth, lat, lon) float32 dask.array<chunksize=(1, 21, 87, 180), meta=np.ndarray>
    temperature_observation_weights  (time, depth, lat, lon) float32 dask.array<chunksize=(1, 21, 87, 180), meta=np.ndarray>
    salinity_observation_weights     (time, depth, lat, lon) float32 dask.array<chunksize=(1, 21, 87, 180), meta=np.ndarray>
    time_bnds                        (time, bnds) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    depth_bnds                       (time, depth, bnds) float32 dask.array<chunksize=(1, 42, 2), meta=np.ndarray>
Attributes: (12/22)
    Conventions:            CF-1.0
    title:                  Temperature and salinity analysis
    DSD_entry_id:           UKMO-L4UHFnd-GLOB-v01
    references:             Website and paper: https://www.metoffice.gov.uk/h...
    institution:            UK Met Office
    contact:                Rachel Killick - rachel.killick@metoffice.gov.uk
    ...                     ...
    southernmost_latitude:  -90.5
    northernmost_latitude:  89.5
    westernmost_longitude:  0.5
    easternmost_longitude:  362.5
    file_quality_index:     0
    licence:                EN4 is distributed under the Non Commercial Gover...

In [13]:
import utils as ut
data_allm=[]
for loc in mooring_vars:
    mdata=mooring_vars[loc]
    mlon=mdata.prof_lon.mean()
    mlat=mdata.prof_lat.mean()
    if mlon<0:
        mlon=360+mlon
    data_mooring=en4_ds.sel(lat=mlat.data,lon=mlon.data,method='nearest')
    data_mooring=data_mooring.interp(depth=mdata.depth)
    data_allm.append(data_mooring)
data_allm=xr.concat(data_allm,'loc')
print('Writing en4 mooring t series to file')
data_allm.to_netcdf(f'../data/mooring_tseries_en4.nc')

Writing en4 mooring t series to file
